In [76]:
import json
import pandas as pd
import os
from pathlib import Path
import sys

sys.path.append("../")


from config import DATA_DIR
from graph_types.prime import PrimeGraph
from graph_types.mag import MagGraph

name = "mag"

if name == "prime":
    graph = PrimeGraph.load()
elif name == "mag":
    graph = MagGraph.load()

/Users/joaquinpolonuer/Documents/software/tesis/zitnik/rags/graphsearch/notebooks/../graph_types/mag.py:83: DtypeWarning: Columns (3,11,12,13,15,16,23,25,29,32,34,35,36,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  nodes_df = pd.read_csv(nodes_file)


In [77]:
logs_dir = DATA_DIR / f"connectedness/{graph.name}_logs_2hop"
json_files = sorted([f for f in logs_dir.glob("*.json")])

data = []

for json_file in json_files:
    with open(json_file, "r") as f:
        log_data = json.load(f)
        

    # Extract key information from each log entry
    record = {
        "file_id": json_file.stem,
        "question": log_data.get("question", ""),
        "sorted_central_nodes": log_data.get("sorted_central_nodes", []),
        "answer_indices": log_data.get("answer_indices", []),
        "sorted_candidates": log_data.get("sorted_candidates", []),
    }

    data.append(record)

df = pd.DataFrame(data)


In [78]:
df["recall@all"] = df.apply(
    lambda row: len(set(row["answer_indices"]).intersection(set(row["sorted_candidates"])))
    / len(set(row["answer_indices"])),
    axis=1,
)
df["hit@1"] = df.apply(
    lambda row: row["sorted_candidates"][0] in row["answer_indices"],
    axis=1,
)
df["hit@5"] = df.apply(
    lambda row: len(set(row["answer_indices"]).intersection(set(row["sorted_candidates"][:5]))) > 0,
    axis=1,
)
df["recall@20"] = df.apply(
    lambda row: len(set(row["answer_indices"]).intersection(set(row["sorted_candidates"][:20])))
    / len(set(row["answer_indices"])),
    axis=1,
)

### Metrics

In [79]:
[
    ("Hit@1", float(round(df["hit@1"].mean(), 3))),
    ("Hit@5", float(round(df["hit@5"].mean(), 3))),
    ("Recall@20", float(round(df["recall@20"].mean(), 3))),
    ("Recall@all", float(round(df["recall@all"].mean(), 3))),
]

[('Hit@1', 0.475),
 ('Hit@5', 0.773),
 ('Recall@20', 0.763),
 ('Recall@all', 0.931)]

### What was the starting node when we didn't hit the correct subgraph?

In [80]:
for _, row in df[df["recall@all"] != 1].iterrows():
    starting_node = row["sorted_central_nodes"][0] if row["sorted_central_nodes"] else "None"
    print(f"Question: {row['question']}\nStarting node: {starting_node}")
    print(
        f"Other candidate for starting node: {row['sorted_central_nodes'][1] if len(row['sorted_central_nodes']) > 1 else 'None'}\n"
    )

Question: Show me research articles related to weather stations focusing on the study of mountainous terrain impacts on observational data.
Starting node: PaperNode(name=Daytime Heat Transfer Processes over Mountainous Terrain, index=1521359, type=paper)
Other candidate for starting node: PaperNode(name=On the Measurement of Turbulence Over Complex Mountainous Terrain, index=1582654, type=paper)

Question: I'm seeking research articles on mathematical objects that delve into delineating complex networks' modular configuration. My focus is on those employing graph theory and algebraic topology approaches to scrutinize the communal constructs within empirical networks.
Starting node: FieldOfStudyNode(name=Algebraic topology, index=1129889, type=field_of_study)
Other candidate for starting node: FieldOfStudyNode(name=Graph theory, index=1123418, type=field_of_study)

Question: Show me publications by M. F. de Andrade that study the interactions of monomers within a lattice structure.
Star

### When we started in the correct subgraph, how many did we recover?

In [81]:
df[df["recall@all"] == 1]['recall@20'].mean()

np.float64(0.8202240933235656)